In [1]:
# Install required libraries
!pip install autograd==1.7.0 autograd-gamma==0.5.0 interface_meta==1.3.0 formulaic==1.0.2 lifelines==0.30.0
!pip install optuna xgboost lightgbm catboost pandas numpy scikit-learn

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from lifelines.utils import concordance_index
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest, f_regression
import optuna
import os

# Step 1: Load and Preprocess Dataset
def load_and_preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Separate numeric and categorical columns
    numeric_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = train_data.select_dtypes(include=["object"]).columns.tolist()

    # Handle missing values
    train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].median())
    test_data = test_data.reindex(columns=train_data.columns, fill_value=0)
    test_data[numeric_cols] = test_data[numeric_cols].fillna(train_data[numeric_cols].median())

    train_data[categorical_cols] = train_data[categorical_cols].fillna(train_data[categorical_cols].mode().iloc[0])
    test_data[categorical_cols] = test_data[categorical_cols].fillna(train_data[categorical_cols].mode().iloc[0])

    # Key columns
    key_columns = ["ID", "efs", "efs_time"]
    key_train_data = train_data[key_columns]
    key_test_data = test_data[["ID"]]

    # One-hot encode categorical variables
    train_data = pd.get_dummies(train_data.drop(columns=key_columns), drop_first=True)
    test_data = pd.get_dummies(test_data.drop(columns=["ID"]), drop_first=True)

    train_data, test_data = train_data.align(test_data, join="left", axis=1)
    test_data.fillna(0, inplace=True)

    # Reattach key columns
    train_data = pd.concat([key_train_data, train_data], axis=1)
    test_data = pd.concat([key_test_data, test_data], axis=1)

    train_data = shuffle(train_data, random_state=42)
    return train_data, test_data

# Step 2: Prepare Features and Targets
def prepare_features_and_targets(train_data, test_data):
    X = train_data.drop(columns=["ID", "efs", "efs_time"])
    y = train_data["efs_time"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    selector = SelectKBest(f_regression, k=min(10, X.shape[1]))
    X_selected = selector.fit_transform(X_scaled, y)

    X_test = test_data.drop(columns=["ID"])
    X_test_scaled = scaler.transform(X_test)
    X_test_selected = selector.transform(X_test_scaled)

    return X_selected, y, X_test_selected, test_data["ID"]

# Step 3: Define Objective Function for Optuna
def objective(trial, X, y):
    model_type = trial.suggest_categorical('model_type', ['xgboost', 'lightgbm', 'catboost'])
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
    }
    if model_type == 'xgboost':
        params['colsample_bytree'] = trial.suggest_uniform('colsample_bytree', 0.6, 1.0)
        params['reg_alpha'] = trial.suggest_loguniform('reg_alpha', 1e-4, 1e-1)
        params['reg_lambda'] = trial.suggest_loguniform('reg_lambda', 1e-4, 1e-1)
        model = XGBRegressor(**params, random_state=42)
    elif model_type == 'lightgbm':
        params['colsample_bytree'] = trial.suggest_uniform('colsample_bytree', 0.6, 1.0)
        params['reg_alpha'] = trial.suggest_loguniform('reg_alpha', 1e-4, 1e-1)
        params['reg_lambda'] = trial.suggest_loguniform('reg_lambda', 1e-4, 1e-1)
        model = LGBMRegressor(**params, random_state=42)
    elif model_type == 'catboost':
        model = CatBoostRegressor(**params, random_state=42, verbose=0)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    c_indices = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        model.fit(X_train, y_train)
        val_predictions = model.predict(X_val)
        c_index = concordance_index(y_val, -val_predictions)
        c_indices.append(c_index)

    return np.mean(c_indices)

# Step 4: Hyperparameter Optimization
def optimize_hyperparameters(X, y, n_trials=100):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X, y), n_trials=n_trials)
    return study.best_params

# Step 5: Train Final Model and Make Predictions
def train_and_predict(X, y, X_test, best_params):
    if best_params['model_type'] == 'xgboost':
        model = XGBRegressor(**best_params, random_state=42)
    elif best_params['model_type'] == 'lightgbm':
        model = LGBMRegressor(**best_params, random_state=42)
    elif best_params['model_type'] == 'catboost':
        model = CatBoostRegressor(**best_params, random_state=42, verbose=0)
    
    model.fit(X, y)
    predictions = model.predict(X_test)
    return predictions

# Main Execution
train_path = "/kaggle/input/equity-post-HCT-survival-predictions/train.csv"
test_path = "/kaggle/input/equity-post-HCT-survival-predictions/test.csv"
train_data, test_data = load_and_preprocess_data(train_path, test_path)
X, y, X_test, test_ids = prepare_features_and_targets(train_data, test_data)

# Hyperparameter Optimization
best_params = optimize_hyperparameters(X, y, n_trials=100)
print("Best parameters:", best_params)

# Train Final Model and Make Predictions
predictions = train_and_predict(X, y, X_test, best_params)

# Step 1: Load the Submission File
submission = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")

# Step 3: Save the Submission File
submission = pd.DataFrame({
    "ID": test_ids.astype(int),
    "prediction": predictions
})
submission.to_csv("/kaggle/working/Submission.csv", index=False)

# Step 4: Print the Shape of Submission File
print("Sub shape:", submission.shape)

# Step 5: Display the First Few Rows
print(submission.head())

# Additional Evaluation Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(X, y, best_params):
    if best_params['model_type'] == 'xgboost':
        model = XGBRegressor(**best_params, random_state=42)
    elif best_params['model_type'] == 'lightgbm':
        model = LGBMRegressor(**best_params, random_state=42)
    elif best_params['model_type'] == 'catboost':
        model = CatBoostRegressor(**best_params, random_state=42, verbose=0)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    mae_scores


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 21.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=feba3a50e9d3f502ae77799451e288ec4b46245dbe463183e55e24481b577550
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


[I 2025-01-28 03:55:31,662] A new study created in memory with name: no-name-213c2b96-68df-424d-ab83-76bfc0b0ace8
<ipython-input-1-fd830db1b355>:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-1-fd830db1b355>:79: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
[I 2025-01-28 03:56:46,596] Trial 0 finished with value: 0.40870480215892435 and parameters: {'model_type': 'catboost', 'n_estimators': 1185, 'learning_rate': 0.0728038718543334, 'max_depth': 11, 'subsample': 0.9850268167513618}. Best is trial 0 with value: 0.40870480215892435.
<ipython-inp

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002869 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 10
[LightGBM] [Info] Start training from score 23.370257
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001728 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 10
[LightGBM] [Info] Start training from score 23.176595
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set:

[I 2025-01-28 03:56:56,514] Trial 1 finished with value: 0.3997043693607437 and parameters: {'model_type': 'lightgbm', 'n_estimators': 1337, 'learning_rate': 0.045569861414372505, 'max_depth': 9, 'subsample': 0.7567890549424783, 'colsample_bytree': 0.937845591696771, 'reg_alpha': 0.007684961882634736, 'reg_lambda': 0.08042738635547002}. Best is trial 0 with value: 0.40870480215892435.
<ipython-input-1-fd830db1b355>:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-1-fd830db1b355>:79: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
<ipython-input-1-

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000950 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 10
[LightGBM] [Info] Start training from score 23.370257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002369 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 10
[LightGBM] [Info] Start training from score 23.370257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001714 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 307
[LightGBM] [I

[I 2025-01-28 04:34:28,431] Trial 78 finished with value: 0.406923217130433 and parameters: {'model_type': 'lightgbm', 'n_estimators': 1332, 'learning_rate': 0.09986996356383494, 'max_depth': 8, 'subsample': 0.6772794546426876, 'colsample_bytree': 0.6550345578870703, 'reg_alpha': 0.005399928236547739, 'reg_lambda': 0.0036540029284460925}. Best is trial 58 with value: 0.4442414545524092.
<ipython-input-1-fd830db1b355>:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-1-fd830db1b355>:79: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
<ipython-input-

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 10
[LightGBM] [Info] Start training from score 23.370257
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001753 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set: 23040, number of used features: 10
[LightGBM] [Info] Start training from score 23.176595
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001890 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 307
[LightGBM] [Info] Number of data points in the train set:

[I 2025-01-28 04:46:57,594] Trial 96 finished with value: 0.4059382320533583 and parameters: {'model_type': 'lightgbm', 'n_estimators': 1476, 'learning_rate': 0.08104152792849521, 'max_depth': 12, 'subsample': 0.6271830899348732, 'colsample_bytree': 0.7875914706472921, 'reg_alpha': 0.006363007184025647, 'reg_lambda': 0.004071561362170131}. Best is trial 92 with value: 0.44447640873860506.
<ipython-input-1-fd830db1b355>:77: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
<ipython-input-1-fd830db1b355>:79: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
<ipython-inpu

Best parameters: {'model_type': 'xgboost', 'n_estimators': 1410, 'learning_rate': 0.09257481429832366, 'max_depth': 12, 'subsample': 0.6135228936223254, 'colsample_bytree': 0.7872598677222539, 'reg_alpha': 0.00643677836424401, 'reg_lambda': 0.002933945497885441}
Sub shape: (3, 2)
      ID  prediction
0  28800   49.039345
1  28801  -24.584087
2  28802   20.959532


In [2]:
import os

# Step 3: Save the Submission File
submission = pd.DataFrame({
    "ID": test_ids.astype(int),
    "prediction": predictions
})

# Save the file and confirm its creation
submission_file_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_file_path, index=False)
print(f"Submission file saved to: {submission_file_path}")

# Check if the file exists
if os.path.exists(submission_file_path):
    print("Submission file exists.")
else:
    print("Error: Submission file does not exist!")

# Display the first few rows of the file
try:
    generated_submission = pd.read_csv(submission_file_path)
    print("Preview of the generated submission file:")
    print(generated_submission.head())
    print(f"Submission file shape: {generated_submission.shape}")
except Exception as e:
    print(f"Error reading the generated submission file: {e}")


Submission file saved to: /kaggle/working/submission.csv
Submission file exists.
Preview of the generated submission file:
      ID  prediction
0  28800   49.039345
1  28801  -24.584087
2  28802   20.959532
Submission file shape: (3, 2)


In [3]:
print(f"Number of rows in test data: {test_data.shape[0]}")
print(f"Length of predictions: {len(predictions)}")
print(f"Length of test_ids: {len(test_ids)}")
# Check test data size
print(f"Number of rows in test data: {test_data.shape[0]}")

# Check predictions and IDs
print(f"Length of predictions: {len(predictions)}")
print(f"Length of test_ids: {len(test_ids)}")

# Debug alignment issues if lengths mismatch
if len(predictions) != len(test_ids):
    print("Error: Mismatch between predictions and test IDs!")


Number of rows in test data: 3
Length of predictions: 3
Length of test_ids: 3
Number of rows in test data: 3
Length of predictions: 3
Length of test_ids: 3


In [4]:
test_data_raw = pd.read_csv(test_path)
print(test_data_raw.head())
print(f"Test dataset shape: {test_data_raw.shape}")


      ID                       dri_score psych_disturb    cyto_score diabetes  \
0  28800  N/A - non-malignant indication            No           NaN       No   
1  28801                    Intermediate            No  Intermediate       No   
2  28802  N/A - non-malignant indication            No           NaN       No   

   hla_match_c_high  hla_high_res_8          tbi_status arrhythmia  \
0               NaN             NaN              No TBI         No   
1               2.0             8.0  TBI +- Other, >cGy         No   
2               2.0             8.0              No TBI         No   

   hla_low_res_6  ... karnofsky_score hepatic_mild          tce_div_match  \
0            6.0  ...            90.0           No                    NaN   
1            6.0  ...            90.0           No  Permissive mismatched   
2            6.0  ...            90.0           No  Permissive mismatched   

  donor_related      melphalan_dose  hla_low_res_8 cardiac  \
0     Unrelated  N/A, M

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
